# Step 2: Loading data to gluon and obtain features of images
** Extract features based on [Gluon Model Zoo](https://mxnet.incubator.apache.org/versions/master/api/python/gluon/model_zoo.html)**

In [1]:
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import image
from mxnet import init
from mxnet import nd
from mxnet.gluon.data import vision
from mxnet.gluon.model_zoo import vision as models
import numpy as np
from tqdm import tqdm


import matplotlib.pyplot as plt
# will make the plot outputs appear and be stored within the notebook.
%matplotlib inline 
# define the wirte lengthy setting
%config InlineBackend.figure_format = 'retina' 

import warnings
warnings.filterwarnings("ignore")

E:\Anaconda\envs\MXNet\lib\site-packages\mxnet-1.0.1-py2.7.egg\mxnet\optimizer.py:136: UserWarning: WARNING: New optimizer mxnet.optimizer.NAG is overriding existing optimizer mxnet.optimizer.NAG
  Optimizer.opt_registry[name].__name__))


**[Image API in MXNet](https://mxnet.incubator.apache.org/api/python/image/image.html)**

**[Gluon Data API](https://mxnet.incubator.apache.org/api/python/gluon/data.html?highlight=imagefolderdataset#mxnet.gluon.data.vision.ImageFolderDataset)**

**[Fine-tuning: 通过微调来迁移学习](http://zh.gluon.ai/chapter_computer-vision/fine-tuning.html)**

2.1 Define the pre-processing function

In [2]:
ctx = mx.gpu()

preprocessing = [
    image.ForceResizeAug((224,224)),
    image.ColorNormalizeAug(mean=nd.array([0.485, 0.456, 0.406]), std=nd.array([0.229, 0.224, 0.225]))
]

def transform(data, label):
    data = data.astype('float32') / 255
    for pre in preprocessing:
        data = pre(data)
    
    data = nd.transpose(data, (2,0,1))
    return data, nd.array([label]).asscalar().astype('float32')

2.2 定义导出特征向量的函数define the output feature

In [3]:
def get_features(net, data):
    features = []
    labels = []

    for X, y in tqdm(data):
        feature = net.features(X.as_in_context(ctx))
        features.append(feature.asnumpy())
        labels.append(y.asnumpy())
    
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    return features, labels

2.3 obtain feature vector

In [4]:
#%%time
preprocessing[0] = image.ForceResizeAug((224,224))
imgs = vision.ImageFolderDataset('for_train',  flag=0, transform=None)
data = gluon.data.DataLoader(imgs, 64)

#features_vgg, labels = get_features(models.vgg16_bn(pretrained=True, ctx=ctx), data)
#features_resnet, _ = get_features(models.resnet152_v1(pretrained=True, ctx=ctx), data)
#features_densenet, _ = get_features(models.densenet161(pretrained=True, ctx=ctx), data)

In [6]:
imgs